In [8]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2
!pip install -U bitsandbytes
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

In [9]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, pipeline, TextGenerationPipeline
import torch

In [10]:
# Models
GPT = 'gpt2'
FALCON = "tiiuae/falcon-rw-1b"
MISTRAL = "mistralai/Mistral-7B-Instruct-v0.1"
Databricks = "databricks/dolly-v2-3b"


In [11]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [12]:
# Flatten the messages into a single plain prompt
# prompt = """
# Generate {{n}} fake job postings for a {{role}} position.

# Only output a JSON array like:
# [
#   {{
#     "title": "Software Engineer",
#     "description": "Develop backend APIs and services.",
#     "requirements": ["Python", "FastAPI", "MongoDB"],
#     "location": "San Francisco",
#     "company_name": "TechCorp"
#   }},
#   ...
# ]
# Return valid JSON only. No markdown. No explanations.
# """

# prompt = """
# Generate exactly {{n}} fake job postings for a {{role}}.

# Each posting must be a JSON object with:
# - title
# - description (5-10 sentences)
# - requirements (array of 3-5 strings)
# - location
# - company_name

# Return a single JSON array with {n} items. No explanations. No markdown.
# ONLY the JSON array as output.
# """

prompt = """
Generate one fake job posting for a {{role}}.

Return only a single JSON object with:
- title
- description (5-10 sentences)
- requirements (array of 4-6 strings)
- location
- company_name

No explanations, no extra text.
Only the JSON object.
"""





In [13]:
!pip install safetensors

In [14]:
import os
os.makedirs("/tmp/dolly_offload", exist_ok=True)

In [15]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [16]:
def load_model_and_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(MISTRAL, trust_remote_code=True)

  model = AutoModelForCausalLM.from_pretrained(
  MISTRAL,
  device_map={"": "cuda"},
  trust_remote_code=True,
  offload_folder="/tmp/dolly_offload",
  quantization_config=bnb_config
  )

  return model, tokenizer


In [17]:
# generator = pipeline("text-generation", model=Databricks, device_map="auto", trust_remote_code=True, offload_folder="/tmp/dolly_offload")

def generate_job(role="Software Engineer", model=None, tokenizer=None):
    # prompt = prompt.format(role=role, n=n)
    # outputs = generator(prompt, max_new_tokens=500, do_sample=True, temperature=0.9)
    # return outputs[0]['generated_text']

    # Apply chat template formatting
    # inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
    inputs = tokenizer(prompt.format(role=role), return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}


    # Generate output
    outputs = model.generate(
        **inputs,
        max_new_tokens=600,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result



In [18]:

def generate_jobs(role="Software Engineer", n=5):
  model, tokenizer = load_model_and_tokenizer()
  role = "Software Engineer"
  fake_jobs = []
  for i in range(n):
      fake_jobs.append(generate_job(role=role, model=model, tokenizer=tokenizer))
  return fake_jobs

In [19]:
print(generate_jobs(role="Software Engineer", n=10))

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

['\nGenerate one fake job posting for a {role}.\n\nReturn only a single JSON object with:\n- title\n- description (5-10 sentences)\n- requirements (array of 4-6 strings)\n- location\n- company_name\n\nNo explanations, no extra text.\nOnly the JSON object.\n\n```json\n{\n  "title": "Software Engineer",\n  "description": "We are looking for a highly skilled software engineer to join our team. The ideal candidate will have experience in developing and maintaining software systems, with a strong understanding of programming languages such as Java or Python. The successful candidate will work collaboratively with our team of developers, designers, and project managers to deliver high-quality software solutions that meet the needs of our clients.",\n  "requirements": ["Bachelor\'s degree in Computer Science or related field", "Minimum of 2 years experience in software development", "Strong understanding of programming languages such as Java or Python", "Experience with software testing and d

In [20]:
!pip install -U bitsandbytes

In [22]:
import re
import json
import ast



def extract_json_objects_from_text_block(texts):
    """
    Accepts either a single string or a list of strings.
    Extracts all valid JSON objects from messy text blocks.
    """
    if isinstance(texts, str):
        texts = [texts]  # wrap in list if single string

    pattern = r"\{[\s\S]*?\}"
    results = []

    for raw_text in texts:
        matches = re.findall(pattern, raw_text)
        for match in matches:
            try:
                obj = json.loads(match)
                results.append(obj)
            except json.JSONDecodeError:
                continue

    return results

text = generate_jobs(role="Software Engineer", n=10)
print(extract_json_objects_from_text_block(text))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'title': 'Software Engineer', 'description': 'We are looking for a highly skilled software engineer to join our team. The ideal candidate will have experience in developing and maintaining complex software systems. Responsibilities include writing clean, efficient, and testable code, collaborating with cross-functional teams, and actively participating in code reviews. Must have a strong understanding of data structures, algorithms, and software design patterns. This is a full-time position located in San Francisco, CA.', 'requirements': ["Bachelor's degree in Computer Science or related field", '3+ years of experience in software development', 'Strong proficiency in Java or C++', 'Experience with agile development methodologies', 'Excellent problem-solving and analytical skills', 'Ability to work independently and as part of a team', 'Strong written and verbal communication skills'], 'location': 'San Francisco, CA', 'company_name': 'Acme Inc.'}, {'title': 'Software Engineer', 'descr

In [23]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.4 MB/s eta 0:00:00


In [25]:
import gradio as gr
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

def generate_ui(role, n):
    try:
        raw_jobs = generate_jobs(role, n)
        parsed_jobs = extract_json_objects_from_text_block(raw_jobs)

        if not isinstance(parsed_jobs, list) or not all(isinstance(item, dict) for item in parsed_jobs):
            print("[ERROR] Parsed result is not a list of dicts")
            return gr.update(value=[], visible=True), None

        filename = f"{role.replace(' ', '_').lower()}_jobs.json"
        with open(filename, "w") as f:
            json.dump(parsed_jobs, f, indent=2)

        print(f"[INFO] Returning {len(parsed_jobs)} jobs -> {filename}")
        return parsed_jobs, filename

    except Exception as e:
        print(f"[FATAL ERROR] {e}")
        return gr.update(value=[], visible=True), None

if __name__ == "__main__":
  with gr.Blocks() as demo:
      gr.Markdown("# 🧠 Synthetic Job Dataset Generator")
      gr.Markdown("Generate a structured dataset of job postings for a specific role.")

      with gr.Row():
          role_input = gr.Textbox(label="Job Role", placeholder="e.g. Software Engineer", value="Software Engineer")
          n_input = gr.Number(label="Number of Samples", value=5, precision=0)

      generate_button = gr.Button("🚀 Generate")
      output_table = gr.JSON(label="Generated Dataset")
      download_button = gr.File(label="Download JSON")

      generate_button.click(
          generate_ui,
          inputs=[role_input, n_input],
          outputs=[output_table, download_button]
      )

  demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bf27145eb99f8caadd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Returning 5 jobs -> software_engineer_jobs.json
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bf27145eb99f8caadd.gradio.live


In [27]:
import os

# Get list of all .ipynb files in /content
notebooks = [f for f in os.listdir("/content") if f.endswith(".ipynb")]
print(notebooks)


[]
